# get different order int addition response from gemini

Install necessary packages

In [1]:
# !pip install -q -U google-generativeai
# !pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
import random
import time
import json
import google.generativeai as genai

### load model

In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
gemini = genai.GenerativeModel('gemini-pro')

### load data and create initial prompt

In [5]:
with open('../../data/diff_order_int_addition.json','r') as file:
    data = json.load(file)

# create batches, batch sizes is heuristics 
# batch size is limited by output token limit which is 2048

size_dict = {'d4':50,
             'd5':50, # big sizes are not working, model gives blank outputs
             'd6':35,
             'd7':35,
             'd8':35,
             'd9':25,
             'd1':25,}


batch_prompts = []

for sign_of_both in data:
    # sign of both : both pos, bignum pos smallnum neg, bignum neg smallnum pos, both neg
    for digit_of_both in data[sign_of_both]:
        # type of both : d41, d42, d43, d51,..d101, d102, d103
        key_to_get_batch_size = digit_of_both[:2]
        list_data = data[sign_of_both][digit_of_both]
        # get batch size
        BATCH_SIZE = size_dict[key_to_get_batch_size]
        
        index = 0

        while index< len(list_data):
            batch_prompt = list_data[index:min(index+BATCH_SIZE,len(list_data))]
            batch_prompts.append(batch_prompt)
            index += BATCH_SIZE
    
print("Number of batches:", len(batch_prompts))
print("Number of prompts in each batch:", [len(batch) for batch in batch_prompts])

Number of batches: 252
Number of prompts in each batch: [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 35, 35, 30, 25, 25, 25, 25, 25, 25, 25, 25,

In [6]:
# give a prompt once and then generate the response,
# and then use this prompt+response directly instead of doing everytime

promt_saying_you_are_a_math_assistant = '''You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 'What is 2 + 3?',you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?'''
Initial_messages = [{'role':'user', 'parts': [promt_saying_you_are_a_math_assistant] }]

# get response from the model and then append it to the prompt
Initial_response = gemini.generate_content(Initial_messages)                                   
Initial_messages.append(Initial_response.candidates[0].content)

In [10]:
print(Initial_messages)

[{'role': 'user', 'parts': ["You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 'What is 2 + 3?',you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?"]}, parts {
  text: "Yes, I understand. I will answer each addition question in the correct format, with each answer on a separate line. For example, if you ask \'What is 2 + 3?\', I will answer \'2 + 3 = 5\'."
}
role: "model"
]


In [11]:
import pickle

# save the initial messages to a file, in case we need to restart the conversation
with open('initial_messages_diff_ord_int_add.pkl', 'wb') as file:
    pickle.dump(Initial_messages, file)

### conduct experiment

In [12]:
# path to save responses
response_file = 'gemini_response/diff_ord_int_addition_response.txt'

# request per minute limit for free api
rpm = 60

In [13]:
# to start from a specific batch number, give printed_batch_num -1 here
batch_no_to_start_from = 0

for i, batch_prompt in enumerate(batch_prompts):
    if i < batch_no_to_start_from:
        continue
    # print status
    print(f"Sending batch {i+1} of {len(batch_prompts)}")

    num_prompts = len(batch_prompt)

    # create prompt for the batch
    curr_prompt = "".join([f"what is {a} + {b} ? \n " for (a,b,c) in batch_prompt])
    
    # create a complete prompt to send to the model
    complete_promt = Initial_messages.copy()
    complete_promt.append({'role':'user', 'parts': [curr_prompt]})

    # get response from the model, candidate_count is 1 because we only need one response,
    # temperature is 0 because we need deterministic responses
    response = gemini.generate_content( complete_promt,
                                        generation_config= genai.GenerationConfig(
                                        candidate_count = 1,
                                        temperature = 0.0,
                                        )
                                )
    
    try:
        # extract text from the response
        model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
    except IndexError:
        print("Error in response")
        model_responses = ["0"]*num_prompts

    # save responses to file
    with open(response_file, 'a') as file:
        for model_response in model_responses:
            file.write(model_response + '\n')

    
    #print the prompt and response separated by space
    print('# of prompts: ',len(batch_prompt),'# of responses: ', len(model_responses))


    for prompt, response in zip(batch_prompt, model_responses):
        print(prompt, response, sep=" ->")
    
    
    print("Received responses for batch", i+1, "of", len(batch_prompts))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    time.sleep(60//rpm) #to avoid rate limit errors

print("Responses saved to", response_file)

Sending batch 1 of 252
# of prompts:  50 # of responses:  50
[7, 3702, 3709] ->7 + 3702 = 3709
[5608, 8, 5616] ->5608 + 8 = 5616
[8776, 9, 8785] ->8776 + 9 = 8785
[9275, 9, 9284] ->9275 + 9 = 9284
[1, 1052, 1053] ->1 + 1052 = 1053
[8015, 2, 8017] ->8015 + 2 = 8017
[4805, 4, 4809] ->4805 + 4 = 4809
[7, 8157, 8164] ->7 + 8157 = 8164
[6, 1558, 1564] ->6 + 1558 = 1564
[2897, 6, 2903] ->2897 + 6 = 2903
[3536, 2, 3538] ->3536 + 2 = 3538
[1320, 8, 1328] ->1320 + 8 = 1328
[7, 6618, 6625] ->7 + 6618 = 6625
[3475, 3, 3478] ->3475 + 3 = 3478
[8699, 4, 8703] ->8699 + 4 = 8703
[4233, 4, 4237] ->4233 + 4 = 4237
[9, 3006, 3015] ->9 + 3006 = 3015
[4, 7357, 7361] ->4 + 7357 = 7361
[9, 3394, 3403] ->9 + 3394 = 3403
[6, 5783, 5789] ->6 + 5783 = 5789
[7, 9215, 9222] ->7 + 9215 = 9222
[7, 8937, 8944] ->7 + 8937 = 8944
[1439, 2, 1441] ->1439 + 2 = 1441
[1, 9126, 9127] ->1 + 9126 = 9127
[2148, 9, 2157] ->2148 + 9 = 2157
[6220, 9, 6229] ->6220 + 9 = 6229
[3, 3960, 3963] ->3 + 3960 = 3963
[6, 6445, 6451] ->6 +